In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Importing the dataset

In [2]:
df= pd.read_csv("Dataset_01.csv")
df

,Unnamed: 0,index,age,job_category,edu,edu_level,marriage,job,rel,race,sex,gain,loss,hours-per-week,nationality,salary
0,0,0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,1,1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,2,2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,3,3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,4,4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,48837,48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,48838,48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,48839,48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,48840,48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Showing ten random rows

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

random_rows = df.sample(n=10)
random_rows

,Unnamed: 0,index,age,job_category,edu,edu_level,marriage,job,rel,race,sex,gain,loss,hours-per-week,nationality,salary
7762,7762,7762,56,Private,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
23881,23881,23881,25,Private,HS-grad,9,Married-civ-spouse,Transport-moving,Own-child,Other,Male,0,0,40,United-States,<=50K
30507,30507,30507,43,Private,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,14344,0,40,United-States,>50K
28911,28911,28911,32,Private,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
19484,19484,19484,39,Private,Bachelors,13,Married-civ-spouse,Sales,Wife,White,Female,0,0,30,United-States,<=50K
43031,43031,43031,20,Private,HS-grad,9,Never-married,Adm-clerical,Unmarried,White,Female,0,0,40,Germany,<=50K
28188,28188,28188,54,Private,HS-grad,9,Divorced,Transport-moving,Not-in-family,White,Male,0,0,45,United-States,<=50K
12761,12761,12761,25,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,1887,40,United-States,>50K
40834,40834,40834,30,Private,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,50,Dominican-Republic,<=50K
27875,27875,27875,33,Private,Some-college,10,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,<=50K


## Since Unnamed:0 and index are identical with the pandas indexing, we can drop them 

In [4]:
df = df.drop(["Unnamed: 0", "index"], axis=1)

delete later

In [5]:
df

,age,job_category,edu,edu_level,marriage,job,rel,race,sex,gain,loss,hours-per-week,nationality,salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   job_category    48842 non-null  object
 2   edu             48842 non-null  object
 3   edu_level       48842 non-null  int64 
 4   marriage        48842 non-null  object
 5   job             48842 non-null  object
 6   rel             48842 non-null  object
 7   race            48842 non-null  object
 8   sex             48842 non-null  object
 9   gain            48842 non-null  int64 
 10  loss            48842 non-null  int64 
 11  hours-per-week  48842 non-null  int64 
 12  nationality     48842 non-null  object
 13  salary          48842 non-null  object
dtypes: int64(5), object(9)
memory usage: 5.2+ MB


## Check if there are Nan values

In [7]:
df.isnull().sum()

age               0
job_category      0
edu               0
edu_level         0
marriage          0
job               0
rel               0
race              0
sex               0
gain              0
loss              0
hours-per-week    0
nationality       0
salary            0
dtype: int64

Transforming object dtypes to string

In [8]:
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].astype("string")

check the counts of each categorical column

In [9]:
for col in df.columns:
    if df[col].dtype == "string":
        print(f"{col}: {df[col].value_counts()}")

job_category: job_category
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: Int64
edu: edu
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: Int64
marriage: marriage
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: count, dtype: Int64
job: job
Prof-specialty       6172
Craft-repair         6112
Exe

## Insights:
- there are ? values rather than Nan in categorical columns
- dataset is binary classified
- US has the majority of nationality category. there are some countries with few counts which increases the cardinality of the column. so it is better to transform nationality into a more big picture class like : US, south america, eastern europe, western europe, and asia
- there is also a South category which is not so correct so I will transform it into ? 
- We can Also put 12th 11th to 7th-8th into one category

In [10]:
# Mapping dictionary for broader categories
nationality_mapping = {
    'United-States': 'US',
    'Mexico': 'NorthAmerica',
    'Canada': 'NorthAmerica',
    'Puerto-Rico': 'NorthAmerica',
    'El-Salvador': 'NorthAmerica',
    'Guatemala': 'NorthAmerica',
    'Cuba': 'NorthAmerica',
    'Jamaica': 'NorthAmerica',
    'Haiti': 'NorthAmerica',
    'Dominican-Republic': 'NorthAmerica',
    'Outlying-US(Guam-USVI-etc)': 'US',
    'South': '?',
    'Columbia': 'SouthAmerica',
    'Peru': 'SouthAmerica',
    'Ecuador': 'SouthAmerica',
    'Nicaragua': 'SouthAmerica',
    'Trinadad&Tobago': 'SouthAmerica',
    'Honduras': 'SouthAmerica',
    'England': 'WesternEurope',
    'Germany': 'WesternEurope',
    'Italy': 'WesternEurope',
    'Poland': 'EasternEurope',
    'Hungary': 'EasternEurope',
    'Portugal': 'WesternEurope',
    'Ireland': 'WesternEurope',
    'France': 'WesternEurope',
    'Scotland': 'WesternEurope',
    'Yugoslavia': 'EasternEurope',
    'Greece': 'WesternEurope',
    'Holand-Netherlands': 'WesternEurope',
    'Philippines': 'Asia',
    'India': 'Asia',
    'China': 'Asia',
    'Vietnam': 'Asia',
    'Taiwan': 'Asia',
    'Iran': 'Asia',
    'Hong': 'Asia',
    'Cambodia': 'Asia',
    'Thailand': 'Asia',
    'Laos': 'Asia',
    'Japan': 'Asia',
    '?': '?'
}

# Map nationality to broader categories
df['nationality'] = df['nationality'].map(nationality_mapping)

# Display the transformed column
df['nationality'].value_counts()

nationality
US               43855
NorthAmerica      1982
Asia               981
?                  972
WesternEurope      651
SouthAmerica       272
EasternEurope      129
Name: count, dtype: int64

In [11]:
# Mapping dictionary for broader categories
education_mapping = {
    'HS-grad': 'HS-grad',
    'Some-college': 'Some-college',
    'Bachelors': 'Bachelors',
    'Masters': 'Masters',
    'Assoc-voc': 'Assoc-voc',
    '11th': 'Less than High School',
    'Assoc-acdm': 'Assoc-acdm',
    '10th': 'Less than High School',
    '7th-8th': 'Less than High School',
    'Prof-school': 'Prof-school',
    '9th': 'Less than High School',
    '12th': 'Less than High School',
    'Doctorate': 'Doctorate'
}

# Map education level to broader categories
df['edu'] = df['edu'].map(education_mapping)

# Display the transformed column
df['edu'].value_counts()

edu
HS-grad                  15784
Some-college             10878
Bachelors                 8025
Less than High School     5569
Masters                   2657
Assoc-voc                 2061
Assoc-acdm                1601
Prof-school                834
Doctorate                  594
Name: count, dtype: int64

In [12]:
salary_mapping = {
    "<=50K": 0 , 
    ">50K":1
}

df['salary'] = df['salary'].map(salary_mapping)

df['salary'].value_counts()

salary
0    37155
1    11687
Name: count, dtype: int64

## Handling missing values

I will try three methods for handling missing values: 
- delete the rows
- impute with mode
- unsupervised imputation using K-means <br>
I will train a random forest on each method then I will evaluate each method using AUC 

let's create a pipeline for sake of ease

In [25]:
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(df.drop(["salary"], axis=1), df["salary"], train_size=0.6, test_size=0.4,random_state=0)
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and X_train[cname].dtype == "string" ]
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

In [14]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
numerical_transformer = MinMaxScaler()

In [16]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [17]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [18]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [34]:
# from imblearn.under_sampling import RandomUnderSampler 
# X=X_valid
# y=y_valid
# rus = RandomUnderSampler(random_state=42)
# X_res, y_res = rus.fit_resample(X, y)

In [35]:
# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['age', 'edu_level', 'gain',
                                                   'loss', 'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['job_category', 'marriage',
                                                   'rel', 'race', 'sex'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [36]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

In [37]:
auc = roc_auc_score(y_valid, preds)
auc

0.8865019421345619

## Data visualization

Visualization of the categorical columns using bar plots

In [ ]:
import matplotlib.pyplot as plt

# Filter categorical columns
categorical_columns = df.select_dtypes(include=['object', 'string'])

# Calculate the number of rows and columns needed for subplots
num_rows = 3
num_cols = 3

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 25))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot bar plots for each categorical column
for i, col in enumerate(categorical_columns):
    df[col].value_counts().plot(kind="bar", ax=axes[i])
    axes[i].set_title(col)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()



- we have an imbalaced dataset


swarm plot 

In [ ]:
# for col in df.columns[:-1]:  # Exclude the label column
#     plt.figure(figsize=(8, 6))
#     sns.swarmplot(x=df["salary"], y=df[col])
#     plt.title(f"swarm plot for {col}")
#     plt.xlabel(col)
#     plt.ylabel("Diagnosis")
#     plt.show()